# Part 1 - General Setup

### Paths

In [1]:
project_dir = '/content/drive/MyDrive/AMLProject/Oscar/'
run_aug_oscar  = project_dir + 'oscar/run_aug_oscar.py'
data_dir       = project_dir + "dataset/coco_caption/"
oscar_ckpt_dir = project_dir + 'base_oscar/'
gcn_ckpt_dir   = project_dir + 'gcn_checkpoints/'

### Set up notebook

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir(project_dir)

Mounted at /content/drive


In [3]:
!cd $project_dir && python setup.py build develop

running build
running build_py
running develop
running egg_info
writing oscar.egg-info/PKG-INFO
writing dependency_links to oscar.egg-info/dependency_links.txt
writing top-level names to oscar.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'oscar.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/oscar.egg-link (link to .)
Adding oscar 0.1.0 to easy-install.pth file

Installed /content/drive/.shortcut-targets-by-id/1MaSTrCq7mlx_JG-_Dv9aCVkhognEiBTV/AMLProject/Oscar
Processing dependencies for oscar==0.1.0
Finished processing dependencies for oscar==0.1.0


Download the libraries

In [1]:
!pip install -r requirements.txt -qqq
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html -qqq
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html -qqq
!pip install torch-geometric -qqq

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
     |████████████████████████████████| 7.9 MB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 2.6 MB/s 
     |████████████████████████████████| 370 kB 5.3 MB/s 
     |████████████████████████████████| 482 kB 38.2 MB/s 
     |████████████████████████████████| 41 kB 425 kB/s 


Import the libraries

In [5]:
from oscar.run_aug_oscar import GCN, BertTokenizer, CaptionTSVDataset, make_data_sampler, create_single_graph
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
import shutil

Check for the GPU

In [6]:
# Check if cuda available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Graph Convolutional Network

## Create a random GCN

In [7]:
gcn_model = GCN(in_features=2054, hidden_features=2054, output_features=2054, num_blocks=1, p_dropout=0.6)
gcn_model.to(device)
gcn_model

GCN(
  (gcn): Sequential(
    (0): GCNConv(2054, 2054)
    (1): GCNConv(2054, 2054)
    (2): GCNConv(2054, 2054)
  )
)

## Pretraining GCN

Load the data

In [43]:
# Load the data
batch_size = 16
tokenizer = BertTokenizer.from_pretrained(oscar_ckpt_dir)
dataset = CaptionTSVDataset(data_dir + 'train.yaml', tokenizer=tokenizer)
sampler = make_data_sampler(dataset, shuffle=True, distributed=False)

train_dataloader = torch.utils.data.DataLoader(
        dataset, num_workers=1, sampler=sampler,
        batch_size=batch_size,
        pin_memory=True)

Instantiate the optimizer

In [8]:
optimizer = torch.optim.AdamW(gcn_model.parameters())

Training Loop

In [9]:
for step, (img_keys, batch) in enumerate(train_dataloader):
  gcn_model.train()
  batch = tuple(t.to(device) for t in batch)
  input_ids = batch[0]
  img_feats = batch[3]
  img_feats_pred = torch.stack([gcn_model(Data(edge_index=create_single_graph(ids, img), x=img).to(device)) for ids, img in 
                                zip(input_ids, img_feats)])

  loss = F.mse_loss(img_feats, img_feats_pred) 
  loss.backward()
  optimizer.step()
  print(f'Step: {step}, Loss: {loss}')

  if loss < 1e-5:
    print('Stopping criterion')
    torch.save(gcn_model.state_dict(), gcn_ckpt_dir + '/gcn.ckpt')
    break

  if step % 10 == 0:
    torch.save(gcn_model.state_dict(), gcn_ckpt_dir + f'/gcn_model_{step}.ckpt')
    

/content/drive/.shortcut-targets-by-id/1MaSTrCq7mlx_JG-_Dv9aCVkhognEiBTV/AMLProject/Oscar/oscar/run_aug_oscar.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.Tensor(features)


Step: 0, Loss: 8.399324417114258
Step: 1, Loss: 0.8979783654212952
Step: 2, Loss: 0.55570387840271
Step: 3, Loss: 0.34144383668899536
Step: 4, Loss: 0.14776070415973663
Step: 5, Loss: 0.07367932796478271
Step: 6, Loss: 0.020406300202012062
Step: 7, Loss: 0.006094050593674183
Step: 8, Loss: 0.001962833572179079
Step: 9, Loss: 0.0007641626871190965
Step: 10, Loss: 2.8260101316845976e-05
Step: 11, Loss: 4.109167548449477e-06
Stopping criterion


Move the pretrained GCN in the base_oscar folder

In [17]:
gcn_old_chpt_file = gcn_ckpt_dir + 'gcn.ckpt'
gcn_new_chpt_file = project_dir + 'base_oscar/gcn.ckpt'
shutil.move(gcn_old_chpt_file, gcn_new_chpt_file)

'/content/drive/MyDrive/AMLProject/Oscar/output_finetune_oscar/gcn.ckpt'

# Part 2 - Finetuning Oscar on the Image Captioning Task

In [14]:
batch = 40
epochs = 50
save_step = 50
train_size = 10000
test_size = 10000
lr = 0.0001
oscar_finetune_dir = project_dir + 'output_finetune_oscar/'

### Finetune Oscar

In [ ]:
!python $run_aug_oscar \
    --model_name_or_path $oscar_ckpt_dir \
    --do_train \
    --skip_gcn \
    --from_oscar \
    --do_lower_case \
    --add_od_labels \
    --learning_rate $lr \
    --per_gpu_train_batch_size $batch \
    --num_train_epochs $epochs \
    --save_steps $save_step \
    --train_size $train_size \
    --data_dir $data_dir \
    --output_dir $oscar_finetune_dir

### Test Oscar

In [12]:
oscar_chpt_dir = oscar_finetune_dir + 'finetune_v4'

In [ ]:
!python $run_aug_oscar \
    --do_test \
    --do_eval \
    --skip_gcn \
    --per_gpu_eval_batch_size $batch \
    --num_beams 5 \
    --max_gen_length 20 \
    --test_size $test_size \
    --data_dir $data_dir \
    --eval_model_dir $oscar_chpt_dir

# Part 3 -  Training Augmented Oscar

In [ ]:
batch = 40
epochs = 50
save_step = 50
train_size = 10000
test_size = 10000
lr = 0.0001
aug_oscar_finetune_dir = project_dir + 'output_finetune_aug_oscar/'

### Train AugOscar

In [ ]:
!python $run_aug_oscar \
    --model_name_or_path $oscar_chpt_dir \
    --do_train \
    --do_lower_case \
    --add_od_labels \
    --learning_rate $lr \
    --per_gpu_train_batch_size $batch \
    --num_train_epochs $epochs \
    --save_steps $save_step \
    --train_size $train_size \
    --data_dir $data_dir \
    --output_dir $aug_oscar_finetune_dir

### Test AugOscar

In [ ]:
aug_oscar_chpt_dir = aug_oscar_finetune_dir + 'finetune_v4'

In [ ]:
!python $run_aug_oscar \
    --do_test \
    --do_eval \
    --per_gpu_eval_batch_size $batch \
    --num_beams 5 \
    --max_gen_length 20 \
    --test_size $test_size \
    --data_dir $data_dir \
    --eval_model_dir $aug_oscar_chpt_dir